# OS Task Replication with GPT4
In this notebook I aim to replicate the results from the AgentBench paper


### 1. Setup
Note: this assumes Docker is installed locally. 

In [2]:
# check if docker is installed
!docker ps

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


In [7]:
# create conda env and install package
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
# build docker images for the os task
!docker pull mysql
!docker pull ubuntu
!docker build -f data/os_interaction/res/dockerfiles/default data/os_interaction/res/dockerfiles --tag local-os/default
!docker build -f data/os_interaction/res/dockerfiles/packages data/os_interaction/res/dockerfiles --tag local-os/packages
!docker build -f data/os_interaction/res/dockerfiles/ubuntu data/os_interaction/res/dockerfiles --tag local-os/ubuntu

Using default tag: latest
latest: Pulling from library/mysql
Digest: sha256:4a4e5e2a19aab7a67870588952e8f401e17a330466ecfc55c9acf51196da5bd0
Status: Image is up to date for mysql:latest
docker.io/library/mysql:latest
Using default tag: latest
latest: Pulling from library/ubuntu
Digest: sha256:3f85b7caad41a95462cf5b787d8a04604c8262cdcdf9a472b8c52ef83375fe15
Status: Image is up to date for ubuntu:latest
docker.io/library/ubuntu:latest
[+] Building 0.0s (0/0)  docker:default
[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.1s (6/6) FINISHED                                 docker:default
 => [internal] load build definition from default                          0.0s
 => => transferring dockerfile: 291B                                       0.0s
 => [internal] load metadata for docker.io/library/ubuntu:latest           0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B            

### 2. Set API Key
Add your OpenAI API key to the confis/agents/openai-chat.yaml file. The line should read 
`Authorization: Bearer YOUR_KEY_HERE`.

### 3. Start task server
This step needs to be done in a _separate_ terminal window. Open a terminal, navigate to the repo directory and run 

`python -m src.start_task --auto-controller --start os-std 2`

Note: leave the terminal open.

### 4. Create assigner config file

In [16]:
import yaml

cfg = {
    "import": "definition.yaml",
    "concurrency": 
        {
        "task": {"os-std": 5},
        "agent": {"gpt-4-0613": 5}
        },
    "assignments": 
        [{"agent": "gpt-4-0613",
        "task": "os-std"
        }],
    "output": "outputs/{TIMESTAMP}" 
    
}

with open('./configs/assignments/custom.yaml', 'w') as outfile:
    yaml.dump(cfg, outfile, default_flow_style=False, sort_keys=False)

### 4. Start assigner
Once you have the task server running, you can start an assigner which coordinates tasks and agents:

In [18]:
!python -m src.assigner --config configs/assignments/custom.yaml

/home/coco/anaconda3/lib/python3.9/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
    Agent: {'text-davinci-002', 'text-davinci-003', 'vicuna-13b', 'vicuna-7b', 'vicuna-33b', 'wizard-30b', 'gpt-3.5-turbo-0613'}
    Task: {'avalon-dev-naive', 'kg-std', 'alfworld-dev', 'dbbench-std', 'kg-dev', 'avalon-dev-single', 'webshop-dev', 'm2w-std', 'cg-std', 'ltp-std', 'webshop-std', 'ltp-dev', 'cg-dev', 'os-dev', 'dbbench-dev', 'm2w-dev', 'alfworld-std'}
creating os-std client...
TaskClient created: os-std (http://localhost:5000/api)
Message: 144 samples remaining.
Agent "gpt-4-0613" needs to run 1 tasks with total 144 samples:
    Task "os-std": 144
Total:   0%|                                            | 0/144 [00:00<?, ?it/s]
                                                                                
Running Count: 0                                                             
Total:   0%|                            

- function to prog create the tasks with some params 
- pushing (non-working) code to a e.g dev branch
